In [63]:
import boto3
import json
brt = boto3.client(service_name='bedrock-runtime')

def infer_llm(text,context=None):
    body = json.dumps({
        "prompt": f"\n\n\
        Human: Using the context key and the value given below, strictly classify the value in one of the \
        following classes. Please respond with the following class labels only i.e PERSON_NAME, ADDRESS, PHONE, OTHER. \
        Class definitions: \n\
        PERSON_NAME: Represents the name of an actual person. NAMES don't include the names of organizations or companies.\n\
        ADDRESS: Reprents the physical address of a person/organization.\n \
        PHONE: Represents a phone number of a person/organization.\n \
        OTHER: Represents something else. \n\n\n\
        Keys: {context if context else 'Unknown'}\n\
        Value: {text}\
        \n\nAssistant:",
        "max_tokens_to_sample": 300,
        "temperature": 0.1,
        "top_p": 0.9,
    })

    modelId = 'anthropic.claude-v2:1'
    accept = 'application/json'
    contentType = 'application/json'

    response = brt.invoke_model(body=body, modelId=modelId, accept=accept, contentType=contentType)

    response_body = json.loads(response.get('body').read())
    return response_body.get('completion')

# text
# print(response_body.get('completion'))

In [86]:
import os
import json

annotations=[]
path='/home/ec2-user/SageMaker/data/FUNSD/dataset/training_data/annotations'
for file in os.listdir(path):
    if '.json' in file :
        with open(f"{path}/{file}") as file_in:
            annotations.append({"file":file,"annotation":json.load(file_in)})

In [77]:
cache={}
from tqdm import tqdm
for annotation in tqdm(annotations):
    for text_box in annotation['annotation']['form']:
        # print(text_box)
        if 'answer' in text_box['label']:            
            contexts=[link for links in text_box['linking'] for link in links if link !=text_box['id']]
            contexts=[annotation['annotation']['form'][entity_id]['text'] for entity_id in contexts]

            if str(contexts) not in cache:                
                result=infer_llm(text_box['text'],contexts)
                if 'OTHER' not in result:
                    # print(f"Text: {text_box['text']} .\n Context: {contexts}")                    
                    # print(result)
                    if 'NAME' in result:
                        text_box['pii_label']="NAME"
                    if 'PHONE' in result:
                        text_box['pii_label']="PHONE"
                    if 'ADDRESS' in result:
                        text_box['pii_label']="ADDRESS"
                # cache[str(contexts)]=result
            else:
                print(f"Cached response {cache[str(contexts)]}")                

100%|██████████| 50/50 [20:09<00:00, 24.19s/it]


In [78]:
count_map={'NAME':0,'PHONE':0, 'ADDRESS':0}
for annotation in tqdm(annotations):
    for text_box in annotation['annotation']['form']:
        if "pii_label" in text_box:
            count_map[text_box['pii_label']]+=1
print(f"Entity stats in the corpus {count_map}")         

100%|██████████| 50/50 [00:00<00:00, 73584.28it/s]

Entity stats in the corpus {'NAME': 127, 'PHONE': 78, 'ADDRESS': 30}


In [79]:
for annotation in tqdm(annotations):
    for text_box in annotation['annotation']['form']:
         if "pii_label" in text_box:
            text_box['label']=text_box['pii_label'].lower()

100%|██████████| 50/50 [00:00<00:00, 73843.38it/s]


In [80]:
for annotation in tqdm(annotations):
    with open(f'{path}/{annotation["file"]}','w') as file_out:
        json.dump(annotation['annotation'],file_out)        

100%|██████████| 50/50 [00:00<00:00, 395.95it/s]


In [ ]:
annotations[0]

In [87]:
for annotation in tqdm(annotations):
    for text_box in annotation['annotation']['form']:
        if text_box['label'] not in {'name','address','phone'}:
            text_box['label'] = "other"
    with open(f'{path}/{annotation["file"]}','w') as file_out:
        json.dump(annotation['annotation'],file_out)        

100%|██████████| 149/149 [00:00<00:00, 405.12it/s]


In [11]:
from PIL import Image, ImageDraw
from IPython.display import display
import json
import os
import time

# Function to visualize textboxes and their labels
def visualize_annotations(annotations_path, image_path):
        # Load annotations for a specific document (change 'document_name.json' to the desired document's JSON file)
    with open(annotations_path) as file_in:        
        for line in file_in:
            annotations = json.loads(line)

    # Extract textbox annotations
    textbox_annotations = annotations['form']

    # Load the image using PIL
    
    img = Image.open(image_path)
    
    # Create ImageDraw object
    draw = ImageDraw.Draw(img)
    
    # Plot textboxes and their labels
    for textbox in textbox_annotations:
        label = textbox['label']
        coords = textbox['box']
        print(coords)
        x, y, width, height = coords['x'], coords['y'], coords['width'], coords['height']
        
        # Draw rectangle patch directly onto the image
        draw.rectangle([(x, y), (x + width, y + height)], outline='red')
        
        # Display label using text annotation
        draw.text((x, y - 15), label, fill='red')
    
    # Display the image using IPython.display
    display(img)

PATH="/home/ec2-user/SageMaker/data/FUNSD/dataset/training_data/"    
# for file in os.listdir(PATH+"images"):
#     if '.png' in file:
# Visualize annotations for a specific document
visualize_annotations(PATH+'images/00093726.png', PATH+'annotations/00093726.json')
        
        

UnicodeDecodeError: 'utf-8' codec can't decode byte 0x89 in position 0: invalid start byte

In [ ]:
import osa